In [12]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

In [3]:
import dagshub
import mlflow
mlflow.set_tracking_uri('https://dagshub.com/mileshsoni/mlops-mini-project.mlflow')
dagshub.init(repo_owner='mileshsoni', repo_name='mlops-mini-project', mlflow=True)

Accessing as mileshsoni

Initialized MLflow to track repo "mileshsoni/mlops-mini-project"

Repository mileshsoni/mlops-mini-project initialized!

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df = normalize_text(df)

In [7]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

C:\Users\miles\AppData\Local\Temp\ipykernel_28268\1076071684.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


In [8]:
mlflow.set_experiment("Bow vs TfIdf")

2025/05/23 10:03:03 INFO mlflow.tracking.fluent: Experiment with name 'Bow vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/def9472140d247dea81182e1507787f1', creation_time=1747974783076, experiment_id='2', last_update_time=1747974783076, lifecycle_stage='active', name='Bow vs TfIdf', tags={}>

In [13]:
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

In [14]:
# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [15]:
# Start the parent run
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                mlflow.sklearn.log_model(model, "model")
                
                # Save and log the notebook
                mlflow.log_artifact('exp2_bow_vs_tfidf.ipynb')
                
                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

2025/05/23 10:05:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7937349397590362
Precision: 0.7846750727449079
Recall: 0.7970443349753694
F1 Score: 0.7908113391984359
🏃 View run LogisticRegression with BoW at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2/runs/bf8c1a1e0c3b41f7a8a06c92bc2d0835
🧪 View experiment at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2


2025/05/23 10:05:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.7942168674698795
Precision: 0.777882797731569
Recall: 0.8108374384236453
F1 Score: 0.79401833092137
🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2/runs/be950c3623604f5db552261bced9bb59
🧪 View experiment at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2


2025/05/23 10:06:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Algorithm: RandomForest, Feature Engineering: TF-IDF
Accuracy: 0.7739759036144578
Precision: 0.7735470941883767
Recall: 0.7605911330049261
F1 Score: 0.7670144063586687
🏃 View run RandomForest with TF-IDF at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2/runs/7e828079ef584773a8b300da776a6d4b
🧪 View experiment at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2


2025/05/23 10:09:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: BoW
Accuracy: 0.7267469879518073
Precision: 0.8010752688172043
Recall: 0.587192118226601
F1 Score: 0.6776577600909608
🏃 View run GradientBoosting with BoW at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2/runs/a1035b418ead45d3a91d3399733ac1ab
🧪 View experiment at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2


2025/05/23 10:10:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: TF-IDF
Accuracy: 0.7243373493975903
Precision: 0.808066759388039
Recall: 0.5724137931034483
F1 Score: 0.6701268742791234
🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2/runs/6f0f77c4bf5e4d28847facb8df68cb31
🧪 View experiment at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2
🏃 View run All Experiments at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2/runs/e9a2d7d37b70484da2ff23678914e07f
🧪 View experiment at: https://dagshub.com/mileshsoni/mlops-mini-project.mlflow/#/experiments/2
